In [1]:
import numpy as np
import pandas as pd
import FdsPy.spar.myspengine as sp
import FdsPy.qe.myqengine as qe

In [2]:
time_series = qe.TimeSeries(start_date = '0M',end_date = '0M')

univ = qe.ScreenUniverse(universe_expr = '(ISAV(FFD_SHR_CLS_AUM(0,ACT,ROLL,USD))=1 AND ISAV(P_PRICE(0,USD))=1  AND UPPERCASE(P_EXCOUNTRY(ISO2))=UPPERCASE("US") AND PROPER(FFD_CLASS_ASSET(TEXT))=PROPER("EQUITY"))=1')
data_items = {'name':"FFD_NAME",
              'price':'P_PRICE(0,USD)',
              'aum':"FFD_SHR_CLS_AUM(0,ACT,ROLL,USD)",
              'lss_ticker':"LSS_TICKER",
              'ffd_bmk_id':"FFD_BMK_ID",
              'ffd_brand':"FFD_BRAND",
              'focus':'FFD_CLASS_FOCUS(TEXT)',
              'segment':'FFD_SEG',
              'niche':'FFD_CLASS_NICHE(TEXT)',
              'class':'FFD_CLASS_ASSET(TEXT)',
              'category':'FFD_CLASS_CAT(TEXT)'
              }

req = qe.Calculation(universe = univ, dates = time_series,data_dict= data_items)
req.query()


In [3]:
df = req.data
df.head()

,DATE,UNIVERSE,name,price,aum,lss_ticker,ffd_bmk_id,ffd_brand,focus,segment,niche,class,category
0,20230831,RMT,Royce Micro-Cap Trust Incorporated,8.980,5.010069e+08,RMT-US,R.2000,Royce,Micro Cap,Equity: U.S. - Micro Cap,Broad-based,Equity,Size and Style
1,20230831,FUNYX,Pioneer Fundamental Growth Fund,30.590,3.814054e+09,None,R.1000G,Pioneer,Large Cap,Equity: U.S. - Large Cap Growth,Growth,Equity,Size and Style
2,20230831,EMF,Templeton Emerging Markets Fund,11.705,2.161835e+08,EMF-US,NOINDEX,Templeton,Total Market,Equity: Emerging Markets - Total Market,Broad-based,Equity,Size and Style
3,20230831,ABCAX,American Aadvantage Funds,20.690,1.386211e+08,None,R.1000V,American Beacon,Total Market,Equity: U.S. - Total Market,Broad-based,Equity,Size and Style
4,20230831,TACWX,Templeton China World Fund,8.410,2.560751e+07,None,NOINDEX,Templeton,Total Market,Equity: China - Total Market,Broad-based,Equity,Size and Style


In [4]:
df=df.dropna().sort_values('aum',ascending = False).head(250)

In [7]:
import pandas as pd

#Spar inputs
doc_name = "PERSONAL:RETURNS_API"
comp_name = 'Cumulative Returns Table'
comp_cat = 'Raw Data / Returns'
bench_prefix = 'SPN'

ports = df['lss_ticker'].to_list()
bench = 'SP50'
ret_type = 'NTR'
freq = 'Daily'
curr = 'USD'
df_all = pd.DataFrame()

for horizon in ['0Y','0Q','0M']:
    startdate = horizon
    enddate = '0D'

    #post call to SPAR api
    spar_df = sp.calc_spar_data(doc_name,comp_name,comp_cat,ports,bench,bench_prefix,ret_type,startdate,enddate,freq,curr)

    df_full = pd.DataFrame()
    for port in list(spar_df.keys())[1:]:
        df_temp = spar_df[port][0]
        df_temp['ticker'] = port
        df_temp.iloc[:,1] = df_temp.iloc[:,1]
        df_temp =df_temp.rename(columns = {df_temp.columns[1]:port})
        df_temp= df_temp.set_index(['Dates'])
        df_full= pd.concat([df_full,df_temp[port]],axis=1)

    df_m = pd.DataFrame(df_full.stack().rename(f'cum_ret_{horizon}')).reset_index()
    df_m = df_m.rename(columns = {'level_0':'DATE',
                                'level_1':'lss_ticker'})
    df_m = df_m.set_index(['DATE','lss_ticker'])
    df_all = pd.concat([df_all,df_m],axis=1)

df_all

Calculation Unit Id:USA-US Failed!!!
Error message : [{'detail': 'Error getting calculation',
 'id': '9d2ea2543250489b965ca49f40ad866f:USA-US'}]
Calculation Unit Id:USA-US Failed!!!
Error message : [{'detail': 'Error getting calculation',
 'id': '6c0dc8cd855c490ea535e64df0e4d153:USA-US'}]
Calculation Unit Id:USA-US Failed!!!
Error message : [{'detail': 'Error getting calculation',
 'id': '99893d4715d446faa363cb8466ebd387:USA-US'}]


cum_ret_0Y  cum_ret_0Q  cum_ret_0M
DATE       lss_ticker                                    
12/30/2022 SWPPX-US      0.000000         NaN         NaN
           JLGMX-US      0.000000         NaN         NaN
           DFQTX-US      0.000000         NaN         NaN
           DFEOX-US      0.000000         NaN         NaN
           DFIEX-US      0.000000         NaN         NaN
...                           ...         ...         ...
09/15/2023 FVADX-US      3.470917   -1.938124   -2.904928
           JAEDX-US     36.293433    0.857148   -1.553784
           SVSPX-US     17.223602    0.330005   -1.211123
           ACIPX-US     -0.040623   -3.172593   -1.293652
           CABDX-US      5.290102    1.983471   -0.161812

[46314 rows x 3 columns]

In [8]:

df = df.set_index(['lss_ticker'])
df_f = df_all.join(df,how='left')
df_f

cum_ret_0Y  cum_ret_0Q  cum_ret_0M      DATE UNIVERSE  \
DATE       lss_ticker                                                          
12/30/2022 SWPPX-US      0.000000         NaN         NaN  20230831    SWPPX   
           JLGMX-US      0.000000         NaN         NaN  20230831    JLGMX   
           DFQTX-US      0.000000         NaN         NaN  20230831    DFQTX   
           DFEOX-US      0.000000         NaN         NaN  20230831    DFEOX   
           DFIEX-US      0.000000         NaN         NaN  20230831    DFIEX   
...                           ...         ...         ...       ...      ...   
09/15/2023 FVADX-US      3.470917   -1.938124   -2.904928  20230831    FVADX   
           JAEDX-US     36.293433    0.857148   -1.553784  20230831    JAEDX   
           SVSPX-US     17.223602    0.330005   -1.211123  20230831    SVSPX   
           ACIPX-US     -0.040623   -3.172593   -1.293652  20230831    ACIPX   
           CABDX-US      5.290102    1.983471   -0.161812  20230831    CABDX   

                                                                name   price  \
DATE       lss_ticker                                                          
12/30/2022 SWPPX-US                             Schwab Capital Trust   69.75   
           JLGMX-US                                 JPMorgan Trust I   59.33   
           DFQTX-US                   DFA US Core Equity 2 Portfolio   30.80   
           DFEOX-US                   DFA US Core Equity 1 Portfolio   34.12   
           DFIEX-US          DFA International Core Equity Portfolio   14.63   
...                                                              ...     ...   
09/15/2023 FVADX-US                   Franklin Value Investors Trust   56.80   
           JAEDX-US            Hancock John Variable Insurance Trust   25.10   
           SVSPX-US                                       SSGA Funds  236.97   
           ACIPX-US    American Century Mid Cap Value Fund - G Class   15.46   
           CABDX-US                           AB Relative Value Fund    6.18   

                                aum ffd_bmk_id         ffd_brand  \
DATE       lss_ticker                                              
12/30/2022 SWPPX-US    7.462342e+10       SP50            Schwab   
           JLGMX-US    4.126916e+10    R.1000G          JPMorgan   
           DFQTX-US    3.097797e+10     R.3000               DFA   
           DFEOX-US    3.047367e+10     R.3000               DFA   
           DFIEX-US    2.985577e+10     991000               DFA   
...                             ...        ...               ...   
09/15/2023 FVADX-US    1.342501e+09    R.2000V          Franklin   
           JAEDX-US    1.333482e+09       SP50      John Hancock   
           SVSPX-US    1.332163e+09       SP50      State Street   
           ACIPX-US    1.328240e+09     R.MIDV  American Century   
           CABDX-US    1.328236e+09    R.1000V          AB Funds   

                              focus                                  segment  \
DATE       lss_ticker                                                          
12/30/2022 SWPPX-US       Large Cap               Equity: U.S.  -  Large Cap   
           JLGMX-US       Large Cap       Equity: Global  - Large Cap Growth   
           DFQTX-US    Total Market            Equity: U.S.  -  Total Market   
           DFEOX-US    Total Market            Equity: U.S.  -  Total Market   
           DFIEX-US    Total Market  Equity: Global Ex-U.S.  -  Total Market   
...                             ...                                      ...   
09/15/2023 FVADX-US       Small Cap         Equity: U.S.  -  Small Cap Value   
           JAEDX-US    Total Market       Equity: U.S. - Total Market Growth   
           SVSPX-US       Large Cap               Equity: U.S.  -  Large Cap   
           ACIPX-US         Mid Cap           Equity: U.S.  -  Mid Cap Value   
           CABDX-US    Total Market      Equity: U.S.  -  Total Market Value   

               

In [10]:
df_f.to_pickle('../../data/Performance Leaders/univ_cumulative_returns_multihorizon_250_20230918.pkl')